<a href="https://colab.research.google.com/github/nithinkamaraj/DeepLearning-/blob/master/Colab_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [1]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 1


In [4]:

# Split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # Find the end of this pattern
        end_ix = i + n_steps
        # Check if we are beyond the dataset
        if end_ix > len(sequences) - 1:
            break
        # Gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# Define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])

# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# Horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# Choose a number of time steps
n_steps = 3

# Convert into input/output
X, y = split_sequences(dataset, n_steps)

# The dataset knows the number of features, e.g., 3
n_features = X.shape[2]
print("no of features",n_features)
# Define model

model = Sequential()
model.add(LSTM(1000, activation='tanh', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(1000, activation='tanh'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

# Fit model
model.fit(X, y, epochs=1000, verbose=0)

# Demonstrate prediction
x_input = array([[70, 75, 145], [80, 85, 165], [90, 95, 185]])  # Matching 3 features
x_input = x_input.reshape((1, n_steps, n_features))

yhat = model.predict(x_input, verbose=0)
print(yhat)


no of features 3
[[ 90.05495  95.05533 184.97177]]


In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint callback
checkpoint = ModelCheckpoint(
    'model_checkpoint.h5',        # Path to save the model checkpoint
    monitor='val_loss',           # Monitor validation loss (you can also use 'accuracy', etc.)
    save_best_only=True,          # Only save the model if it improves
    verbose=1,                    # Print messages when saving the model
    mode='min'                    # Save when the validation loss is minimized
)

# Train the model and use the checkpoint callback
model.fit(X_train, y_train,
          epochs=2000,
          batch_size=32,
          validation_data=(X_val, y_val),
          callbacks=[checkpoint])


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'model_checkpoint.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
model.compile(optimizer='adam', loss='mse')

# Continue training for another 1000 epochs
model.fit(X_train, y_train, epochs=2000, batch_size=32, initial_epoch=1000, verbose=1)
